# Quadratically Constrained Quadratic Programs
The latest version of this notebook is available on https://github.com/Qiskit/qiskit-iqx-tutorials.

- Introduce Quadratic Programs
    - Mathematical model (incl. constraints) + variable types<br>
    https://en.wikipedia.org/wiki/Quadratically_constrained_quadratic_program
    - How to build it with docplex (brief example)
    - How to build the same with the QuadraticProgram
- Example
  - How to make an optimization model for some problems

We briefly introduce how to make an optimization model with Aqua optiomization stack.
You can find detailed information at [Qiskit Aqua API references](https://qiskit.org/documentation/apidoc/optimization/optimization.html).

Aqua optimization stack introduces `QuadraticProgram` class to make a model of an optimization problem.
More precicely, it deals with quadratically constrained quadratic program as follows.
$$
\begin{align}
\text{minimize}\quad& x^\top Q_0 x + c^\top x\\
\text{subject to}\quad& A x \leq b\\
& x^\top Q_i x + a_i^\top x \leq r_i, \quad 1,\dots,i,\dots,q\\
& l_i \leq x_i \leq u_i, \quad 1,\dots,i,\dots,n,
\end{align}
$$
where $Q_i$ and $A$ are $n \times n$ matrices, $x$, $b$, and $c$ are $n$-dimensional vectors.

As setup, you need to import the following module.

In [1]:
from qiskit.optimization import QuadraticProgram

You start with a empty model or load a problem from files of particular formats such as LP and MPS.
You can display your optimization problem with a method `pprint_as_string` and `print_as_lp_string`.

In [2]:
# load an LP file
mod = QuadraticProgram()
mod.read_from_lp_file('aux_files/sample.lp')
print(mod.pprint_as_string())

// This file has been generated by DOcplex
// model name is: sample
// single vars section
dvar bool x;
dvar int y;
dvar float z;

minimize
 x - y + 10 z [ 0.500000 x^2 - y*z ];
 
subject to {
 lin_eq:
  x + 2 y == 1;
 lin_leq:
  x + 2 y <= 1;
 lin_geq:
  x + 2 y >= 1;
 quad_eq:
  [ x^2 - y*z + 2 z^2 ] + x + y == 1;
 quad_leq:
  [ x^2 - y*z + 2 z^2 ] + x + y <= 1;
 quad_geq:
  [ x^2 - y*z + 2 z^2 ] + x + y >= 1;

}



In [3]:
print(mod.print_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: sample

Minimize
 obj: x - y + 10 z + [ x^2 - 2 y*z ]/2
Subject To
 lin_eq: x + 2 y = 1
 lin_leq: x + 2 y <= 1
 lin_geq: x + 2 y >= 1
 quad_eq: [ x^2 - y*z + 2 z^2 ] + x + y = 1
 quad_leq: [ x^2 - y*z + 2 z^2 ] + x + y <= 1
 quad_geq: [ x^2 - y*z + 2 z^2 ] + x + y >= 1

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



Aqua optimization stack supports the conversion from Docplex model. You can easily make a model of optimization problem with Docplex.
You can find the documentation of Docplex at https://ibmdecisionoptimization.github.io/docplex-doc/mp/index.html

You can load a Docplex model to `QuadraticProgram` by invoking `from_docplex`.

In [4]:
# Make a Docplex model
from docplex.mp.model import Model
from docplex.mp.linear import Var

mdl = Model('docplex model')
x = mdl.binary_var('x')
y = mdl.integer_var(lb=-1, ub=5, name='y')
mdl.minimize(x + 2 * y)
mdl.add_constraint(x - y == 3)
mdl.add_constraint((x + y) * (x - y) <= 1)
print(mdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex model

Minimize
 obj: x + 2 y
Subject To
 c1: x - y = 3
 qc1: [ x^2 - y^2 ] <= 1

Bounds
 0 <= x <= 1
 -1 <= y <= 5

Binaries
 x

Generals
 y
End



In [5]:
# load from a Docplex model
mod = QuadraticProgram()
mod.from_docplex(mdl)
print(mod.print_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex model

Minimize
 obj: x + 2 y
Subject To
 c0: x - y = 3
 q0: [ x^2 - y^2 ] <= 1

Bounds
 0 <= x <= 1
 -1 <= y <= 5

Binaries
 x

Generals
 y
End



We then explain how to make model of an optimization problem directly using `QuadraticProgram`.
Let's start from an empty model.

In [6]:
# make an empty problem
mod = QuadraticProgram('my problem')
print(mod.print_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj:
Subject To

Bounds
End



Aqua optimization stack supports three types of variables:
- Binary varible
- Integer variable
- Continuous variable

You can specify nemes, types, lower bounds and upper bounds.

When you display your problem as LP format,
`Binaries` denotes binary variables and `Generals` denotes integer variables.
If variables are not included in either `Binaries` or `Generals`, such variables are continuous ones.

Note that you cannot use 'e' or 'E' as the first character of names due to the [specification of LP format](https://www.ibm.com/support/knowledgecenter/SSSA5P_12.7.1/ilog.odms.cplex.help/CPLEX/FileFormats/topics/LP_VariableNames.html).

In [7]:
# Add variables
mod.binary_var(name='x')
mod.integer_var(name='y', lowerbound=-1, upperbound=5)
mod.continuous_var(name='z', lowerbound=-1, upperbound=5)
print(mod.print_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj:
Subject To

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



You can set the objective function by invoking `QuadraticProgram.minimize` or `QuadraticProgram.maximize`.
You can add linear and quadratic objective function by specifying linear and quadratic terms with either matrix or dictionary.

In [8]:
# Add objective function

mod.minimize(constant=3, linear={'x': 1}, quadratic={('x', 'y'): 2, ('z', 'z'): -1})
print(mod.print_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj: x + [ 4 x*y - 2 z^2 ]/2 + 3
Subject To

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



In [9]:
# Add objective function

mod.minimize(constant=3, linear=[1,0], quadratic=[[0,1],[1,-1]])
print(mod.print_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj: x + [ 4 x*y - 2 y^2 ]/2 + 3
Subject To

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



You can access the constant, the linear term, and the quadratic term by looking at `Quadratic.objective.{constant, linear, quadratic}`, respectively.
As for linear and quadratic terms, you can get matrix and dictionary.

In [10]:
print('constant:\t\t', mod.objective.constant)
print('linear:\t\t\t', mod.objective.linear.to_array())
print('quadratic w/ index:\t', mod.objective.quadratic.to_dict())
print('quadratic w/ name:\t', mod.objective.quadratic.to_dict(use_name=True))

constant:		 3
linear:			 [1 0]
quadratic w/ index:	 {(0, 1): 2.0, (1, 1): -1.0}
quadratic w/ name:	 {('x', 'y'): 2.0, ('y', 'y'): -1.0}


You can add linear constraints by setting name, linear expression, sense and right-hand-side value (rhs).
You can use senses 'EQ', 'LE', and 'GE' as Docplex supports.

In [11]:
# Add linear constraints
mod.linear_constraint(linear={'x': 1, 'y': 2}, sense='==', rhs=3, name='lin_eq')
mod.linear_constraint(linear={'x': 1, 'y': 2}, sense='<=', rhs=3, name='lin_leq')
mod.linear_constraint(linear={'x': 1, 'y': 2}, sense='>=', rhs=3, name='lin_geq')
print(mod.print_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj: x + [ 4 x*y - 2 y^2 ]/2 + 3
Subject To
 lin_eq: x + 2 y = 3
 lin_leq: x + 2 y <= 3
 lin_geq: x + 2 y >= 3

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



You can add quadratic constraints as well as objective function and linear constraints.

In [12]:
# Add quadratic constraints
mod.quadratic_constraint(linear={'x': 1, 'y': 1}, quadratic={('x', 'x'): 1, ('y', 'z'): -1}, sense='==', rhs=1, name='quad_eq')
mod.quadratic_constraint(linear={'x': 1, 'y': 1}, quadratic={('x', 'x'): 1, ('y', 'z'): -1}, sense='<=', rhs=1, name='quad_leq')
mod.quadratic_constraint(linear={'x': 1, 'y': 1}, quadratic={('x', 'x'): 1, ('y', 'z'): -1}, sense='>=', rhs=1, name='quad_geq')
print(mod.print_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj: x + [ 4 x*y - 2 y^2 ]/2 + 3
Subject To
 lin_eq: x + 2 y = 3
 lin_leq: x + 2 y <= 3
 lin_geq: x + 2 y >= 3
 quad_eq: [ x^2 - y*z ] + x + y = 1
 quad_leq: [ x^2 - y*z ] + x + y <= 1
 quad_geq: [ x^2 - y*z ] + x + y >= 1

Bounds
 0 <= x <= 1
 -1 <= y <= 5
 -1 <= z <= 5

Binaries
 x

Generals
 y
End



You can substitue some of variables with constants or other variables.
More precicely, `QuadraticProgram` has a method `substitute_variables(constants=..., variables=...)` to deal with the following two cases.
- $x \leftarrow c$: when `constants` have a dictionary `{x: c}`. 
- $x \leftarrow c y$: when `variables` have a dictuinary `{x: (y, c)}`.

In [13]:
sub, status = mod.substitute_variables(constants={'x': 0}, variables={'y': ('z', -1)})
print('status', status)
print(sub.print_as_lp_string())

status SubstitutionStatus.success
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj: [ - 2 z^2 ]/2 + 3
Subject To
 lin_eq: - 2 z = 3
 lin_leq: - 2 z <= 3
 lin_geq: - 2 z >= 3
 quad_eq: [ z^2 ] - z = 1
 quad_leq: [ z^2 ] - z <= 1
 quad_geq: [ z^2 ] - z >= 1

Bounds
 -1 <= z <= 1
End



If the resulting problem is infeasible due to lower bounds or upper bounds, the methods returns the status `SubstitutionStatus.infeasible`.
We try to replace variable `x` with -1, but -1 is out of range of `x` (0 <= `x` <= 1). So, it returns `SubstitutionStatus.infeasible`.

In [14]:
sub, status = mod.substitute_variables(constants={'x': -1})
print('status', status)

Infeasible substitution for variable: x


status SubstitutionStatus.infeasible


You cannot substitute a variables multiple times. The metdod raise an error in such a case.

In [15]:
sub, status = mod.substitute_variables(constants={'x': -1}, variables={'y': ('x', 1)})

QiskitOptimizationError: 'Cannot substitute by variable that gets substituted itself: y <- x 1'